In [ ]:
pip install poliastro

In [ ]:
pip install pyswarms

In [ ]:
import numpy as np
import pyswarms as ps

from pyswarms.utils.functions import single_obj as fx

from pyswarms.single.global_best import GlobalBestPSO

In [ ]:
def rosenbrock_with_args(x, a, b, c=0):
  f = (a - x[:, 0])**2 + b * (x[:, 1] - x[:, 0] ** 2) ** 2 + c
  return f

x_max = 10 * np.ones(2)
x_min = -1 * x_max

bounds = (x_min, x_max)

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = GlobalBestPSO(n_particles=10, dimensions=2, options=options, bounds=bounds)

cost, pos = optimizer.optimize(rosenbrock_with_args, 2000, a=1, b=100, c=0)

print(pos)


2023-08-01 20:00:11,374 - pyswarms.single.global_best - INFO - Optimize for 2000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|2000/2000, best_cost=0
2023-08-01 20:00:20,483 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [1. 1.]


[1. 1.]


In [ ]:
import astropy

from astropy import units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

solar_system_ephemeris.set("jpl")

import numpy as np

from poliastro.bodies import Sun, Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.twobody import Orbit
from poliastro.util import norm, time_range

from poliastro.iod.izzo import lambert
from poliastro.threebody.flybys import compute_flyby
from poliastro.threebody.soi import laplace_radius

from scipy.optimize import bisect

import matplotlib.pyplot as plt

In [ ]:
def quad(x, a, b):
  return a * x**2 + b

def find_brackets(func, low, high, args=()):
  tests = np.linspace(low, high, 10000)
  a = low
  b = high

  a_found = False
  b_found = False

  for test in tests:
    if np.sign(func(test, *args)) == -1 and not a_found:
      a = test
      a_found = True
    elif np.sign(func(test, *args)) == 1 and not b_found:
      b = test
      b_found = True
    elif a_found and b_found:
      print('a is', a)
      print('b is', b)
      break

  return min(a, b), max(a, b)

print(find_brackets(quad, -100, 100, (-2, 2)))

a is -100.0
b is -0.9900990099009874
(-100.0, -0.9900990099009874)


In [ ]:
###### DIRECTIONS CELL
## Goal 1 is to create a function that can accurately take:
#### launch date
#### times of flight along each leg
#### flyby sequence
## and return accurate
#### total delta v
#### total cost of the flight
#### list of maneuver delta vs
#### list of maneuver dates
#### list of radii of periapse of the flybys

date_launch = Time("2011-08-05 16:25", scale="utc").tdb
date_flyby = Time("2013-10-09 19:21", scale="utc").tdb
date_arrival = Time("2016-07-05 03:18", scale="utc").tdb

# launch_date should be a string as above in UTC
# flight_times should be an np array (1D) and in JD
# flyby sequence should be an np array (1D)

# launch energy from an Atlas V rocket
#C3 = 31.1 * u.km**2 / u.s**2

def find_brackets(func, low, high, args=()):
  tests = np.linspace(low, high, 1000000)
  #ys = [func(test, *args) for test in tests]

  #plt.plot(tests, ys)
  #plt.show()
  a = low
  b = high

  a_found = False
  b_found = False

  for test in tests:
    if np.sign(func(test, *args)) == -1 and not a_found:
      a = test
      a_found = True
    elif np.sign(func(test, *args)) == 1 and not b_found:
      b = test
      b_found = True
    elif a_found and b_found:
      #print('a is', a)
      #print('b is', b)
      break

  return min(a, b), max(a, b)


def rp_to_turn_difference(rp_guess, turn_angle, v_inf_in, v_inf_out, k):
  mag_vinf_in = norm(v_inf_in)
  mag_vinf_out = norm(v_inf_out)

  t_nondim = np.radians(turn_angle.value)
  mvin_nondim = mag_vinf_in.value
  mvout_nondim = mag_vinf_out.value
  k_nondim = k.value

  e_in = 1 + rp_guess * mvin_nondim**2 / k_nondim
  e_out = 1 + rp_guess * mvout_nondim**2 / k_nondim

  turn_difference = np.arcsin(1/e_in) + np.arcsin(1/e_out) - t_nondim
  return turn_difference

# make sure that a is in km and e is u.one
def compute_insertion_dv(attractor, desired_a, desired_e, v_inf_in):
  mag_vin = norm(v_inf_in)
  k_planet = getattr(attractor, 'k').to(u.km**3 / u.s**2)

  rp_desired = desired_a * (1 - desired_e)
  vp_in = np.sqrt(mag_vin**2 + 2 * k_planet / rp_desired)

  vp_desired = np.sqrt(k_planet * (2 / rp_desired - 1 / desired_a))

  return np.abs(vp_in - vp_desired)

def compute_collision_cost(flyby_planet, rp):
  radius = getattr(flyby_planet, 'R')
  parenth = rp / (1.05 * radius)

  dec = -2 * np.log10(parenth)

  if dec < 0:
    score = 0
  else:
    score = dec

  return score

def compute_hyperbolic_cost(flyby_planet, v_inf_in):
  mag_vin = norm(v_inf_in)

  k_planet = getattr(flyby_planet, 'k').to(u.km**3 / u.s**2)
  soi = laplace_radius(flyby_planet)

  decision = (0.9 * mag_vin)**2 / 2 - k_planet / soi

  if decision < 0:
    score = 1 / mag_vin
  else:
    score = 0

  return score


####################################
####################################
####################################
####################################
####################################


# fix launch_date to be part of np.array with flight_times
# launch_date should be entered into the np.array as JD value (not a time obj, just a scalar)
# then, launch_date should be changed into ISOT date for date_launch obj to be made
def analyze_trajectory(flight_times, flyby_sequence):
  #print(flight_times)
  launch_date = flight_times[0]
  flight_times = flight_times[1:]
  date_launch = Time(launch_date, scale="utc", format='jd').tdb

  jds = np.zeros(len(flyby_sequence))
  jds[0] = date_launch.jd
  for i in range(1, len(jds)):
    temp_jd = jds[i-1] + flight_times[i-1]
    jds[i] = temp_jd
  #print(jds)

  dates = Time(jds, scale='utc', format='jd')

  date_arrival = dates[-1]

  mercury = Ephem.from_body(Mercury, time_range(date_launch, end=date_arrival, periods=5000), attractor=Sun)
  venus = Ephem.from_body(Venus, time_range(date_launch, end=date_arrival, periods=5000), attractor=Sun)
  earth = Ephem.from_body(Earth, time_range(date_launch, end=date_arrival, periods=5000), attractor=Sun)
  mars = Ephem.from_body(Mars, time_range(date_launch, end=date_arrival, periods=5000), attractor=Sun)
  jupiter = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival, periods=5000), attractor=Sun)
  saturn = Ephem.from_body(Saturn, time_range(date_launch, end=date_arrival, periods=5000), attractor=Sun)
  uranus = Ephem.from_body(Uranus, time_range(date_launch, end=date_arrival, periods=5000), attractor=Sun)
  neptune = Ephem.from_body(Neptune, time_range(date_launch, end=date_arrival, periods=5000), attractor=Sun)

  planet_dict = {1:mercury, 2:venus, 3:earth, 4:mars, 5:jupiter, 6:saturn, 7:uranus, 8:neptune}
  body_dict = {1:Mercury, 2:Venus, 3:Earth, 4:Mars, 5:Jupiter, 6:Saturn, 7:Uranus, 8:Neptune}

  # since we are launching from earth:

  # assuming that insertion velocity is tangential to that of the Earth

  # it is a good assumption that initial orbit is same as Earth
  #initial_orbit = Orbit.from_ephem(Sun, earth, date_launch)

  #sc_earth_initial =

  flyby_planets = flyby_sequence[1:]

  k_sun = getattr(Sun, 'k').to(u.km**3 / u.s**2)

  v_sc_outs = []
  v_sc_ins = []

  for i in range(1, len(flyby_sequence)):
    prev_planet = flyby_sequence[i-1]
    prev_planet_str = planet_dict[prev_planet]
    prev_date = dates[i-1]

    next_planet = flyby_planets[i-1]
    next_planet_str = planet_dict[next_planet]
    next_date = dates[i]

    #print(prev_planet)
    #print(next_planet)

    r0, v0_body = prev_planet_str.rv(prev_date)
    #print(r0, v0_body.to(u.km/u.s))
    r, v_body = next_planet_str.rv(next_date)

    tof = (next_date - prev_date)
    tof_seconds = tof.to(u.s)

    v0_sc, v_sc = lambert(k_sun, r0, r, tof_seconds)
    #print(v0_sc)
    #print(v_sc)
    #print()

    v_sc_outs.append(v0_sc)
    v_sc_ins.append(v_sc)
  #print(len(v_sc_outs))

  home_planet = flyby_sequence[0]
  home_planet_str = planet_dict[home_planet]
  home_date = dates[0]
  home_r, home_v = home_planet_str.rv(home_date)
  home_v = home_v.to(u.km/u.s)
  launch_dv = norm(home_v - v_sc_outs[0])

  dvs = [launch_dv]
  rps = []
  #print(launch_dv)
  total_dv = 0
  total_dv += launch_dv

  total_gscore = 0

  #print(flyby_planets)
  for j in range(0, len(flyby_planets) - 1):
    v_sc_in = v_sc_ins[j]
    v_sc_out = v_sc_outs[j+1]

    flyby_planet = flyby_planets[j]
    flyby_planet_str = planet_dict[flyby_planet]
    flyby_date = dates[j+1]
    #print(flyby_date)
    flyby_pl_r, flyby_pl_v = flyby_planet_str.rv(flyby_date)
    flyby_pl_v = flyby_pl_v.to(u.km/u.s)

    #print(v_sc_in)
    #print(v_sc_out)

    v_inf_in = v_sc_in - flyby_pl_v
    v_inf_out = v_sc_out - flyby_pl_v
    #print(v_inf_in)
    #print(flyby_pl_v)

    #print(norm(v_inf_in))
    #print(norm(v_inf_out))

    turn_angle = np.arccos(np.dot(v_inf_in, v_inf_out) / (norm(v_inf_in) * norm(v_inf_out)))
    turn_angle = np.degrees(turn_angle)

    #print(turn_angle)

    flyby_planet_obj = body_dict[flyby_planet]
    flyby_planet_k = getattr(flyby_planet_obj, 'k').to(u.km**3 / u.s**2)

    #print(flyby_planet_k)

    bound_a, bound_b = find_brackets(rp_to_turn_difference, 0, 15000000, args=(turn_angle, v_inf_in, v_inf_out, flyby_planet_k))


    rp = bisect(rp_to_turn_difference, bound_a, bound_b, args=(turn_angle, v_inf_in, v_inf_out, flyby_planet_k))
    #print(rp)
    print('passed rp section')

    rp = rp * u.km
    rps.append(rp)

    mag_vin = norm(v_inf_in)
    mag_vout = norm(v_inf_out)

    dvp1 = np.sqrt(mag_vin**2 + 2 * flyby_planet_k / rp)
    dvp2 = np.sqrt(mag_vout**2 + 2 * flyby_planet_k / rp)

    dv = np.abs(dvp1 - dvp2)

    total_dv += dv
    dvs.append(dv)

    collision_g = compute_collision_cost(flyby_planet_obj, rp)
    hyperbolic_g = compute_hyperbolic_cost(flyby_planet_obj, v_inf_in)

    total_gscore += (collision_g + hyperbolic_g)

  a = 176338 * u.km
  e = 0.998 * u.one
  dest_planet = flyby_sequence[-1]
  dest_planet_obj = body_dict[dest_planet]
  dest_planet_str = planet_dict[dest_planet]
  insertion_date = dates[-1]
  dest_r, dest_v = dest_planet_str.rv(insertion_date)
  dest_v = dest_v.to(u.km/u.s)

  v_inf_in = v_sc_ins[-1] - dest_v

  insertion_dv = compute_insertion_dv(dest_planet_obj, a, e, v_inf_in)
  total_dv += insertion_dv
  dvs.append(insertion_dv)

  total_score = total_dv.value + total_gscore

  return total_score, total_dv, dates, dvs, rps

def analyze_trajectory_give_score(flight_times, flyby_sequence):
  scores = np.zeros((len(flight_times), 1))
  for i in range(0, len(flight_times)):
    flight_times_particle = flight_times[i]
    total_score, _, _, _, _ = analyze_trajectory(flight_times_particle, flyby_sequence)
    scores[i][0] = total_score
  print('reached end of first it')
  return scores











test_seq = np.array([3, 2, 2, 3, 3, 5])
#test_launch = "1988-04-09 00:00"
test_times = np.array([[2447260.5006503, 175, 437, 57, 628, 1081]])

print(analyze_trajectory_give_score(test_times, test_seq))








passed rp section
passed rp section
passed rp section
passed rp section
reached end of first it
[[3.92474022]]


In [ ]:
import pyswarms as ps

from pyswarms.utils.functions import single_obj as fx

from pyswarms.single.global_best import GlobalBestPSO

In [ ]:
def inner_loop(flyby_sequence, launch_lower, launch_upper):
  low_bounds = [launch_lower]
  high_bounds = [launch_upper]
  body_info = {1:(87.969, 57.989e6), 2:(224.7, 108.2e6), 3:(365.2, 149.6e6), 4:(687.0, 228.0e6), 5:(4331, 778.5e6), 6:(10747, 1432.0e6), 7:(30589, 2867.0e6), 8:(59800, 4515.0e6)}

  for i in range(0, len(flyby_sequence) - 1):
    p1 = flyby_sequence[i]
    p2 = flyby_sequence[i+1]

    if p1 == p2:
      period = body_info[p1][0]
      low_bound = period / 2
      high_bound = 5 * period
    else:
      period1 = body_info[p1][0]
      a1 = body_info[p1][1]

      period2 = body_info[p2][0]
      a2 = body_info[p2][1]

      if max(a1, a2) < 2.992e8:
        low_bound = 0.1 * min(period1, period2)
        high_bound = 1.5 * max(period1, period2)
      else:
        low_bound = 0.1 * min(period1, period2)
        high_bound = max(period1, period2)

    if low_bound > 600:
      low_bound = 600
    if high_bound < 1000:
      high_bound = 1000

    low_bounds.append(low_bound)
    high_bounds.append(high_bound)


  ##
  low_bounds = np.array(low_bounds)
  high_bounds = np.array(high_bounds)

  bounds = (low_bounds, high_bounds)
  options = {'c1': 0.65, 'c2': 2.0, 'w': 2.0}
  dims = len(low_bounds)

  optimizer = GlobalBestPSO(n_particles=20*dims, dimensions=dims, options=options, bounds=bounds)

  cost, pos = optimizer.optimize(analyze_trajectory_give_score, 2000, flyby_sequence=flyby_sequence)

  return cost, pos








test_seq = np.array([3, 2, 2, 3, 3, 5])
cost, pos = inner_loop(test_seq, 2446431.5, 2449353.5)

print('Optimal cost is:')
print(cost)
print()

print('Optimal dates are:')
print(pos)
print()






2023-08-01 20:00:54,226 - pyswarms.single.global_best - INFO - Optimize for 2000 iters with {'c1': 0.65, 'c2': 2.0, 'w': 2.0}
pyswarms.single.global_best:   0%|          |0/2000

passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp section
passed rp 

pyswarms.single.global_best:   0%|          |0/2000


ValueError: ignored